In [135]:
import pandas as pd
from plotly import express as px
import numpy as np

In [136]:
# import zillow home value index for all homes (SFR, Condo/Co-op) for US cities
df_zillow = pd.read_csv('../City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
df_zillow.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,2000-01-31,2000-02-29,...,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31
0,6181,0,New York,city,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,130287.519134,130986.454069,...,624591.374167,632503.861776,639719.068154,644183.951334,645330.342375,642250.704367,639868.002388,636974.535334,632470.800784,629516.303341
1,12447,1,Los Angeles,city,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,205857.114461,206148.139291,...,899748.915585,908394.728386,913205.457773,915884.082386,913542.541439,909121.015865,904309.731512,901908.970409,898585.043168,890194.164509
2,39051,2,Houston,city,TX,TX,"Houston-The Woodlands-Sugar Land, TX",Harris County,95018.228813,94992.795115,...,247356.386226,251261.832612,254553.486054,256833.443059,257899.901884,258099.225384,258069.554808,258056.821868,257327.171539,255921.337783


In [137]:
df_zillow.info(show_counts=True)

df_zillow.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22390 entries, 0 to 22389
Columns: 285 entries, RegionID to 2023-01-31
dtypes: float64(277), int64(2), object(6)
memory usage: 48.7+ MB


RegionID         0
SizeRank         0
RegionName       0
RegionType       0
StateName        0
              ... 
2022-09-30    1029
2022-10-31    1030
2022-11-30    1030
2022-12-31    1032
2023-01-31      18
Length: 285, dtype: int64

In [138]:
# change home values from float to int & fill NA with 0
df_zillow.iloc[:, 8:] = df_zillow.iloc[:, 8:].fillna(0).astype(int)

# drop excess columns:
df_zillow.drop(['RegionID', 'StateName', 'RegionType', 'Metro', 'CountyName'], axis=1, inplace=True)

# rename RegionName to City
df_zillow.rename(columns={'RegionName': 'City'}, inplace=True)

df_zillow.head(3)

,SizeRank,City,State,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,...,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31
0,0,New York,NY,130287,130986,131695,133068,134445,135927,137481,...,624591,632503,639719,644183,645330,642250,639868,636974,632470,629516
1,1,Los Angeles,CA,205857,206148,207040,208814,211046,213210,215431,...,899748,908394,913205,915884,913542,909121,904309,901908,898585,890194
2,2,Houston,TX,95018,94992,94860,94818,94800,94959,95156,...,247356,251261,254553,256833,257899,258099,258069,258056,257327,255921


In [139]:
# df for median home values by state for Jan 2023
df_state_Jan_2023 = df_zillow.groupby('State').agg({'2023-01-31': 'median'}).reset_index()
px.scatter(df_state_Jan_2023, y='2023-01-31', x='State', title= 'Median Home Value for Jan 2023 by State', labels={'2023-01-31':'Home Value in $'})

HI has the highest median home value, followed by DC and CA. It's expected that states with expensive cities and less overall cities have higher median values than states with many cities with a broad range of home values. Next, we'll explore the number of cities in each state.

In [140]:
df_city_count_by_state = df_zillow.groupby(['State']).agg({'City': 'count'}).sort_values('City').reset_index()
df_city_count_by_state.rename(columns={'City': 'Number of Cities'}, inplace=True)
df_city_count_by_state.head(10)

,State,Number of Cities
0,DC,1
1,AK,43
2,RI,43
3,NV,52
4,DE,58
5,HI,68
6,WY,94
7,NM,135
8,SD,139
9,ND,141


DC which has one expensive city has one of the highest state median home value.

In [141]:
# create new df for the top 100 largest cities for analysis
df_largerst_100_cities = df_zillow.iloc[:100,:]
# histogram for typical home value in top 100 largest cities for Jan 2023
fig = px.histogram(df_largerst_100_cities, x='City', y='2023-01-31', color='State', title='Median Home Value: 100 Largest US Cities for Jan 2023', labels={'2023-01-31':'Home Value in $'})
fig.for_each_trace(lambda t: t.update(hovertemplate=t.hovertemplate.replace("sum of", "")))
fig.for_each_yaxis(lambda a: a.update(title_text=a.title.text.replace("sum of", "")))

There's a lot of variability in home values across the largest 100 cities, for example typical home in San Francisco, CA is about 1.3M while it's 0.1M in Toledo, OH. From the histogram, we can see that CA has the most large and expensive cities, followed by WA, and several states that only have one large and expsive city.

Conclusions:
- 
- 
- 
- 